# Pesquisa CBGG 2025

## Import de bibliotecas

In [3]:
from pysus import SIH
from pysus.ftp import FTPSingleton
import pandas as pd
import source.graphfile as gf

## Download e carregamento dos Dados

In [2]:
FTPSingleton.timeout = 180
print("Carregando arquivos do SIH...")
sih = SIH().load()
print("Arquivos do SIH carregados!")

files = sih.get_files("RD", uf="CE",
year=[2014,2015,2016,2017,2018,2019,2020,2021,2022,2023],
month=[1,2,3,4,5,6,7,8,9,10,11,12])

print("Baixando arquivos do SIH...")
arquivos = sih.download(files,local_dir="./dados/DBC")
print("Arquivos do SIH baixados!")
print("Gerando dataframe dos dados do SIH...")
dataframes = []
for arquivo in arquivos:
    df = arquivo.to_dataframe()
    dataframes.append(df)
    print(f"df {arquivo} gerado!")
df_sih = pd.concat(dataframes, ignore_index=True)
df_sih.to_csv("dados/dados_sih.csv")
print("Datafrane do SIH gerado!")


Carregando arquivos do SIH...
Arquivos do SIH carregados!
Baixando arquivos do SIH...


3259775it [00:00, 180316351.09it/s]    

Arquivos do SIH baixados!
Gerando dataframe dos dados do SIH...


df dados/DBC/RDCE1401.parquet gerado!
df dados/DBC/RDCE1402.parquet gerado!
df dados/DBC/RDCE1403.parquet gerado!
df dados/DBC/RDCE1404.parquet gerado!
df dados/DBC/RDCE1405.parquet gerado!
df dados/DBC/RDCE1406.parquet gerado!
df dados/DBC/RDCE1407.parquet gerado!
df dados/DBC/RDCE1408.parquet gerado!
df dados/DBC/RDCE1409.parquet gerado!
df dados/DBC/RDCE1410.parquet gerado!
df dados/DBC/RDCE1411.parquet gerado!
df dados/DBC/RDCE1412.parquet gerado!
df dados/DBC/RDCE1501.parquet gerado!
df dados/DBC/RDCE1502.parquet gerado!
df dados/DBC/RDCE1503.parquet gerado!
df dados/DBC/RDCE1504.parquet gerado!


KeyboardInterrupt: 

In [ ]:
cnes_mapping = pd.read_csv("dados/CNESBR.csv")
munic_mapping = pd.read_csv("dados/MUNICSBR.csv")
df = pd.read_csv("dados/dados_sih.csv")

# Garantir que os códigos estão no mesmo formato (string)
df["CNES"] = df["CNES"].astype(str)
df["MUNIC_RES"] = df["MUNIC_RES"].astype(str)
cnes_mapping["CNES"] = cnes_mapping["CNES"].astype(str)
munic_mapping["COD"] = munic_mapping["COD"].astype(str)

# Substituir os códigos de CNES pelos nomes
df = df.merge(cnes_mapping.rename(columns={"NOMEFANT": "HOSPITAL"}), how="left", on="CNES")

# Substituir os códigos de MUNIC_RES pelos nomes
df = df.merge(munic_mapping.rename(columns={"MUNIC": "MUNICIPIO", "COD": "MUNIC_RES"}), how="left", on="MUNIC_RES")

# Exibir algumas linhas para verificar
print(df.head())

## Filtro dos Dados

In [ ]:
df["IDADE"] = pd.to_numeric(df["IDADE"], errors="coerce")
# TODO: Levantar a diferença dos dados de Municipios Fora e Dentro do Estado do Ceará
# TODO: Incluir idade de pacientes Centenários
df_s72 = df[
    df["DIAG_PRINC"].isin(["S720", "S721", "S722"]) & 
    df["MUNICIPIO"].str.endswith(", CE") & 
    (df["IDADE"] >= 60)
]
df_s72 = df_s72.dropna(subset=["HOSPITAL", "MUNICIPIO"])
df_s72

,Unnamed: 0,UF_ZI,ANO_CMPT,MES_CMPT,ESPEC,CGC_HOSP,N_AIH,IDENT,CEP,MUNIC_RES,...,TPDISEC2,TPDISEC3,TPDISEC4,TPDISEC5,TPDISEC6,TPDISEC7,TPDISEC8,TPDISEC9,HOSPITAL,MUNICIPIO
3278,3278,230190,2014,1,1,3.284505e+12,2314101285628,1,63180970,230190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOSPITAL MATERNIDADE SAO VICENTE DE PAULO HMSVP,"BARBALHA, CE"
4821,4821,230250,2014,1,1,NaN,2314101326790,1,63275000,230720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IMTAVI,"JATI, CE"
4883,4883,230250,2014,1,1,NaN,2314101326251,1,63270000,231110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IMTAVI,"PORTEIRAS, CE"
4884,4884,230250,2014,1,1,NaN,2314101326262,1,63250000,230830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IMTAVI,"MILAGRES, CE"
4886,4886,230250,2014,1,1,NaN,2314101326284,1,63275000,230720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IMTAVI,"JATI, CE"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5011761,5011761,230440,2023,12,3,7.835044e+12,2323107568408,1,60741450,230440,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,IJF INSTITUTO DR JOSE FROTA CENTRAL,"FORTALEZA, CE"
5011974,5011974,230440,2023,12,1,7.835044e+12,2323107519777,1,60760415,230440,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,IJF INSTITUTO DR JOSE FROTA CENTRAL,"FORTALEZA, CE"
5012636,5012636,230440,2023,12,1,4.885197e+12,2323107685118,1,60130400,230440,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HOSPITAL DISTRITAL MARIA JOSE BARROSO DE OLIVE...,"FORTALEZA, CE"
5013814,5013814,230530,2023,12,3,NaN,2323103146980,1,62360000,230530,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HOSPITAL MUNICIPAL MARIA WANDERLENE NEGREIROS ...,"IBIAPINA, CE"


## Plotar Grafo

In [5]:
# Carregar os dados geográficos
geo_data = pd.read_csv("./dados/geograf.csv")
grafoS72 = gf.criar_grafo(geo_data, df_s72)
gf.plotar_grafo_folium(grafoS72,output_path='./output/grafoS72.html')

## Gerar caracterização dos dados

In [6]:
# Raça (ordenado crescentemente pela categoria)
raca = (df_s72["RACA_COR"].value_counts(normalize=True) * 100).round(1).reset_index()
raca.columns = ["Raça", "Percentual (%)"]
raca = raca.sort_values(by="Raça")  # Ordenar pela coluna "Raça"

# Sexo (ordenado crescentemente por categoria)
sexo = (df_s72["SEXO"].value_counts(normalize=True) * 100).round(1).reset_index()
sexo.columns = ["Sexo", "Percentual (%)"]
sexo = sexo.sort_values(by="Sexo")  # Ordenar pela coluna "Sexo"

# Estatísticas de Idade (não precisa de ordenação, apenas exibição)
df_s72["IDADE"] = pd.to_numeric(df_s72["IDADE"], errors="coerce")  # Converter para numérico
idade_stats = pd.DataFrame({
    "Estatística": ["Mínima", "Máxima", "Média", "Mediana", "Moda"],
    "Idade": [
        int(df_s72["IDADE"].min()),
        int(df_s72["IDADE"].max()),
        round(df_s72["IDADE"].mean(), 1),
        int(df_s72["IDADE"].median()),
        int(df_s72["IDADE"].mode()[0])
    ]
})

# Faixa Etária (<60, 60-69, 70-79, 80-89, 90-99, 100+)
bins = [0, 59, 69, 79, 89, 99, float("inf")]
labels = ["<60", "60-69", "70-79", "80-89", "90-99", "100+"]
df_s72["FAIXA_ETÁRIA"] = pd.cut(df_s72["IDADE"], bins=bins, labels=labels, ordered=True)
faixa_etaria = (df_s72["FAIXA_ETÁRIA"].value_counts(normalize=True) * 100).round(1).reset_index()
faixa_etaria.columns = ["Faixa Etária", "Percentual (%)"]
faixa_etaria = faixa_etaria.sort_values(by="Faixa Etária")  # Ordenar pela coluna "Faixa Etária"

# Exibir as tabelas organizadas
print("Tabela de Raça:")
print(raca)

print("\nTabela de Sexo:")
print(sexo)

print("\nEstatísticas de Idade:")
print(idade_stats)

print("\nTabela de Faixa Etária:")
print(faixa_etaria)

Tabela de Raça:
   Raça  Percentual (%)
2     1             6.8
4     2             1.0
0     3            64.4
3     4             4.4
5     5             0.0
1    99            23.4

Tabela de Sexo:
   Sexo  Percentual (%)
1     1            26.4
0     3            73.6

Estatísticas de Idade:
  Estatística  Idade
0      Mínima   60.0
1      Máxima   99.0
2       Média   79.0
3     Mediana   80.0
4        Moda   82.0

Tabela de Faixa Etária:
  Faixa Etária  Percentual (%)
4          <60             0.0
2        60-69            16.4
1        70-79            32.9
0        80-89            38.8
3        90-99            11.8
5         100+             0.0


In [7]:
# Carregar o arquivo TB_SIGTAW.csv que contém os códigos e nomes dos procedimentos
tb_sig = pd.read_csv("dados/TB_SIGTAW.csv")

# Renomear as colunas de TB_SIGTAW.csv para facilitar o merge
tb_sig.rename(columns={"IP_COD": "PROC_REA", "IP_DSCR": "PROCEDIMENTO"}, inplace=True)

# Garantir que os tipos de dados sejam consistentes para o merge (caso necessário)
df_s72["PROC_REA"] = df_s72["PROC_REA"].astype(str)
tb_sig["PROC_REA"] = tb_sig["PROC_REA"].astype(str)

# Realizar o merge para adicionar a coluna 'PROCEDIMENTO'
df_s72 = df_s72.merge(tb_sig, on="PROC_REA", how="left")

frequencia = df_s72["PROCEDIMENTO"].value_counts()
percentual = df_s72["PROCEDIMENTO"].value_counts(normalize=True) * 100

tabela_frequencias = pd.DataFrame({
    'Frequência': frequencia,
    'Percentual (%)': percentual
})

tabela_frequencias.to_excel("output/freq_procedimentos.xlsx")
print(tabela_frequencias)

                                                    Frequência  Percentual (%)
PROCEDIMENTO                                                                  
TRATAMENTO CIRURGICO DE FRATURA TRANSTROCANTERIANA        5024       40.311322
ARTROPLASTIA PARCIAL DE QUADRIL                           1504       12.067720
TRATAMENTO CIRURGICO DE FRATURA SUBTROCANTERIANA          1384       11.104870
ARTROPLASTIA TOTAL PRIMARIA DO QUADRIL NAO CIME...         938        7.526278
TRATAMENTO CIRURGICO DE FRATURA / LESAO FISARIA...         779        6.250501
TRATAMENTO C/ CIRURGIAS MULTIPLAS                          732        5.873385
ARTROPLASTIA TOTAL PRIMARIA DO QUADRIL CIMENTADA           636        5.103105
TRATAMENTO CLINICO/CONSERVADOR DE TRAUMATISMOS ...         500        4.011875
TRATAMENTO CIRURGICO EM POLITRAUMATIZADO                   289        2.318864
TRATAMENTO CONSERVADOR DE FRATURA / LESAO LIGAM...         270        2.166413
DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CLI...  

In [14]:
df2 = df[
    df["DIAG_PRINC"].isin(["S720", "S721", "S722"]) & 
    df["MUNICIPIO"].str.endswith(", CE")
]
df2["COD_IDADE"].value_counts()

COD_IDADE
4    15993
5       64
3        3
2        2
Name: count, dtype: int64

In [15]:
frequencia = df2["PROCEDIMENTO"].value_counts()
percentual = df2["PROCEDIMENTO"].value_counts(normalize=True) * 100

tabela_frequencias2 = pd.DataFrame({
    'Frequência': frequencia,
    'Percentual (%)': percentual
})

# tabela_frequencias.to_excel("output/freq_procedimentos.xlsx")
print(tabela_frequencias2)

KeyError: 'PROCEDIMENTO'